In [ ]:
print("Memulai proses setup...")

# Pindah ke direktori konten utama
%cd /content

# Clone repository ComfyUI dan custom node
print("Cloning repository ComfyUI dan custom nodes...")
!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI -q
!git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF -q

# Pindah ke direktori ComfyUI
%cd /content/ComfyUI

print("Menginstal dependensi yang diperlukan...")
!pip install -r requirements.txt accelerate torchsde gradio gguf -q
print("Instalasi dependensi selesai.")

# Instal aria2 untuk mempercepat unduhan
!apt install aria2 -qqy

# Download model-model yang diperlukan menggunakan aria2
print("Mengunduh model Unet...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-T2V-A14B-LowNoise-Q3_K_S.gguf -d /content/ComfyUI/models/unet -o Wan2.2-T2V-A14B-LowNoise-Q3_K_S.gguf

print("Mengunduh model CLIP Encoder...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/city96/umt5-xxl-encoder-gguf/resolve/main/umt5-xxl-encoder-Q3_K_S.gguf -d /content/ComfyUI/models/clip -o umt5-xxl-encoder-Q3_K_S.gguf

print("Mengunduh model VAE...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors -d /content/ComfyUI/models/vae -o wan_2.1_vae.safetensors

print("Mengunduh model LoRA...")
!mkdir -p /content/ComfyUI/models/loras/FusionX
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vrgamedevgirl84/Wan14BT2VFusioniX/resolve/main/FusionX_LoRa/Wan2.1_T2V_14B_FusionX_LoRA.safetensors -d /content/ComfyUI/models/loras/FusionX -o Wan2.1_T2V_14B_FusionX_LoRA.safetensors

print("Setup selesai!")

In [ ]:
print("\nMemulai inisialisasi Python...")

# --- Instalasi library yang dibutuhkan jika belum ada ---
# nest_asyncio diperlukan untuk menjalankan asyncio di dalam Colab/Jupyter
try:
    import nest_asyncio
except ImportError:
    print("Menginstal nest_asyncio...")
    !pip install -q nest_asyncio
    import nest_asyncio

import sys
import torch
import random
import time
from PIL import Image
import numpy as np
import gradio as gr
import os
import asyncio

# Terapkan patch asyncio agar bisa berjalan di dalam event loop Colab
nest_asyncio.apply()

# --- Navigasi Direktori ---
# Pastikan kita berada di direktori yang benar
try:
    os.chdir('/content/ComfyUI')
    print(f"Berpindah ke direktori: {os.getcwd()}")
except FileNotFoundError:
    print(f"Sudah berada di direktori {os.getcwd()} atau direktori ComfyUI tidak ditemukan.")


# --- Pemuatan Asynchronous Custom Nodes ---
# Import dilakukan di dalam fungsi async untuk memastikan lingkungan sudah siap
async def load_custom_nodes():
    """
    Memuat semua custom node yang dibutuhkan secara asynchronous.
    """
    print("Memulai pemuatan custom nodes...")
    from nodes import load_custom_node
    # Path harus absolut atau relatif dari direktori kerja saat ini
    custom_node_path = os.path.join(os.getcwd(), "custom_nodes", "ComfyUI-GGUF")
    if not os.path.isdir(custom_node_path):
        print(f"!!! WARNING: Direktori custom node tidak ditemukan di {custom_node_path}")
        return
    await load_custom_node(custom_node_path)
    print("Custom nodes berhasil dimuat.")

# Jalankan pemuatan custom node menggunakan asyncio.run() yang sekarang aman digunakan
# berkat nest_asyncio.
print("Menjalankan pemuatan custom node...")
asyncio.run(load_custom_nodes())


# --- Import dan Inisialisasi Model Utama ---
# Import class setelah custom node dipastikan telah dimuat
from nodes import NODE_CLASS_MAPPINGS, KSamplerAdvanced, VAEDecode, CLIPTextEncode, EmptyLatentImage, VAELoader, LoraLoaderModelOnly
import comfy.samplers
from comfy_extras.nodes_model_advanced import ModelSamplingSD3

print("Menginisialisasi class-class node...")
try:
    UnetLoaderGGUF = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
    CLIPLoaderGGUF = NODE_CLASS_MAPPINGS["CLIPLoaderGGUF"]()
except KeyError as e:
    print(f"FATAL ERROR: Gagal menemukan node '{e.args[0]}'.")
    print("Pastikan custom node 'ComfyUI-GGUF' terinstal dengan benar di direktori 'custom_nodes' dan telah dimuat.")
    sys.exit(1)

# Inisialisasi node bawaan
vae_loader = VAELoader()
lora_loader = LoraLoaderModelOnly()

print("\nMemuat model utama (Unet, CLIP, VAE)...")
try:
    unet_model = UnetLoaderGGUF.load_unet("Wan2.2-T2V-A14B-LowNoise-Q3_K_S.gguf")[0]
    clip_model = CLIPLoaderGGUF.load_clip("umt5-xxl-encoder-Q3_K_S.gguf", "wan")[0]
    vae_model = vae_loader.load_vae("wan_2.1_vae.safetensors")[0]

    print("Memuat model LoRA...")
    lora_model = lora_loader.load_lora_model_only(unet_model, "FusionX/Wan2.1_T2V_14B_FusionX_LoRA.safetensors", 1.0)[0]
    print("Semua model berhasil dimuat!")
except Exception as e:
    print(f"!!! FATAL ERROR SAAT MEMUAT MODEL: {e} !!!")
    print("Pastikan nama file model dan path-nya sudah benar dan file tersebut ada di direktori 'ComfyUI/models/'.")
    import traceback
    traceback.print_exc()
    sys.exit(1)


# --- Fungsi Generate Gambar ---
def generate_image(positive_prompt, negative_prompt, width, height, steps, cfg, seed, sampler_name, scheduler):
    """
    Fungsi utama untuk menghasilkan gambar berdasarkan input dari antarmuka Gradio.
    """
    print("\n--- Memulai Proses Generate Gambar ---")
    start_time = time.time()

    try:
        # Inisialisasi node yang dibutuhkan di dalam fungsi untuk state yang bersih
        clip_text_encode = CLIPTextEncode()
        empty_latent_image = EmptyLatentImage()
        k_sampler_advanced = KSamplerAdvanced()
        vae_decode = VAEDecode()
        model_sampler_patcher = ModelSamplingSD3()

        with torch.inference_mode():
            print("-> Melakukan encoding pada prompt...")
            positive_conditioning = clip_text_encode.encode(clip_model, positive_prompt)[0]
            negative_conditioning = clip_text_encode.encode(clip_model, negative_prompt)[0]

            print("-> Melakukan patch pada model dengan sampler...")
            model_with_sampler = model_sampler_patcher.patch(lora_model, 1.0)[0]

            print(f"-> Membuat latent kosong dengan ukuran {width}x{height}...")
            latent_image = empty_latent_image.generate(width, height, 1)[0]

            if int(seed) == -1:
                seed = random.randint(0, 2**64 - 1)
            print(f"-> Menggunakan Seed: {seed}")

            print(f"-> Menjalankan KSampler dengan {steps} steps...")
            samples = k_sampler_advanced.sample(
                model=model_with_sampler, add_noise="enable", noise_seed=int(seed), steps=int(steps),
                cfg=cfg, sampler_name=sampler_name, scheduler=scheduler,
                positive=positive_conditioning, negative=negative_conditioning, latent_image=latent_image,
                start_at_step=0, end_at_step=9999, return_with_leftover_noise="disable"
            )[0]

            print("-> Melakukan decode pada hasil sample menjadi gambar...")
            decoded_image_tensor = vae_decode.decode(vae_model, samples)[0]

            # Konversi tensor ke PIL Image
            image_np = decoded_image_tensor.cpu().numpy()
            image_np_uint8 = (image_np.clip(0, 1) * 255).astype(np.uint8)
            final_image = Image.fromarray(image_np_uint8[0])

            end_time = time.time()
            print(f"--- Proses Generate Gambar Selesai (Durasi: {end_time - start_time:.2f} detik) ---")

            return final_image

    except Exception as e:
        print(f"!!! TERJADI ERROR SAAT GENERATE GAMBAR: {e} !!!")
        traceback.print_exc()
        # Mengembalikan gambar kosong atau placeholder jika terjadi error
        return Image.new('RGB', (512, 512), color = 'red')


# --- Antarmuka Gradio ---
print("\nMenyiapkan antarmuka Gradio...")

with gr.Blocks(css="footer {display: none !important}") as demo:
    gr.Markdown("## ComfyUI GGUF Image Generator")
    with gr.Row():
        with gr.Column(scale=3):
            positive_prompt = gr.Textbox(label="Positive Prompt", lines=3, placeholder="Contoh: a cinematic photo of a majestic lion...")
            negative_prompt = gr.Textbox(label="Negative Prompt", lines=3, value="blurry, low quality, static, frame, text, watermark")
            with gr.Row():
                sampler_name = gr.Dropdown(label="Sampler", choices=comfy.samplers.KSampler.SAMPLERS, value="euler_ancestral")
                scheduler = gr.Dropdown(label="Scheduler", choices=comfy.samplers.KSampler.SCHEDULERS, value="normal")
            with gr.Row():
                width = gr.Slider(label="Width", minimum=256, maximum=1920, value=1024, step=64)
                height = gr.Slider(label="Height", minimum=256, maximum=1080, value=576, step=64)
            with gr.Row():
                steps = gr.Slider(label="Steps", minimum=1, maximum=20, value=10, step=1)
                cfg = gr.Slider(label="CFG Scale", minimum=0.0, maximum=15.0, value=3.5, step=0.5)
            seed = gr.Number(label="Seed (-1 untuk acak)", value=-1, precision=0)
            generate_button = gr.Button("Generate", variant="primary")

        with gr.Column(scale=2):
            output_image = gr.Image(label="Generated Image", type="pil", format="png", height=512)

    inputs = [positive_prompt, negative_prompt, width, height, steps, cfg, seed, sampler_name, scheduler]
    generate_button.click(fn=generate_image, inputs=inputs, outputs=[output_image], api_name="generate")

print("\nMeluncurkan antarmuka Gradio...")
demo.launch(share=True, inline=False, debug=True)